In [3]:
import os
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
mpl.rcParams["figure.dpi"] = 300
from IPython.display import HTML, clear_output
from PIL import Image

plt.style.use("dark_background")
import glob
import re

import anndata
import h5py
import joblib
import pandas as pd
import psutil
import pymeshfix
import pyvista as pv
import scanpy as sc
import skimage
from morphometrics.measure import measure_selected
from skimage.measure import label
from tqdm import tqdm

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 350
import h5py
from morphometrics.data import cylinders_and_spheres
from morphometrics.explore.cluster import cluster_features
from morphometrics.explore.dimensionality_reduction import pca, umap
from morphometrics.measure import measure_selected
from morphometrics.utils.anndata_utils import table_to_anndata

rng = np.random.default_rng(42)
from morphology.annotator import create_labels, plot_save
from morphology.cell_extractor import read_extract

In [5]:
# extract cells + read morphometrics from the segmentations
seg_directory = "/cluster/project/treutlein/DATA/imaging/viventis/Morphodynamics_of_human_early_brain_organoid_development/3D_Brain_organoids_half_res_morphometrics/"
experiment_directories = [
    "AGAR_replicates_v2_all_06_02_2023/",
    "images_AGAR_one_per_day_all_images_lamin_all_06_02_2023/",
    "3D_one_image_per_day_AGAR_all_all_06_02_2023/",
    "3D_one_image_per_day_all_06_02_2023/",
]
from joblib import Parallel, delayed

all_structures = pd.DataFrame()
for experiment_directory in experiment_directories:
    input_dir = seg_directory + experiment_directory
    print(input_dir)
    output_dir = input_dir + "/morphometrics/"
    all_files = [os.path.basename(x) for x in glob.glob(f"{input_dir}/images/*")]
    results = Parallel(n_jobs=12, verbose=10)(
        delayed(read_extract)(i) for i in range(len(all_files))
    )
    for result in tqdm(results):
        all_structures = all_structures.append(result)
    del results

In [18]:
# remove measurements with nans
region_properties_table = all_structures
region_properties_table = region_properties_table.reset_index()
region_properties_table = region_properties_table.dropna(axis=1, how="all")
region_properties_table = region_properties_table.dropna(axis=0, how="any")
# Remove unlikely masks
region_properties_table["z_hight"] = (
    region_properties_table["bbox-3"] - region_properties_table["bbox-0"]
)
region_properties_table = region_properties_table.loc[
    region_properties_table["max_intensity"] >= 20
]
region_properties_table = region_properties_table.loc[
    (region_properties_table["area"] >= 100 / (0.347 * 2 * 2 * 0.347 * 2 * 0.347))
]

new_tab = (
    region_properties_table.loc[:, region_properties_table.columns != "time_point"]
    .iloc[:, 2:-6]
    .copy()
)

all_data = table_to_anndata(measurement_table=new_tab)
# Use PCA to reduce dimensionality
plt.style.use("default")

%matplotlib inline

pca(all_data, normalize_data=True)
sc.pl.pca(all_data)
sc.pl.pca_variance_ratio(all_data)
# Leiden clustering
cluster_features(
    all_data,
    method="leiden",
    compute_neighbors=True,
    neighbors_kwargs={"n_pcs": 6},
    clustering_kwargs={"resolution": 0.8},
)

region_properties_table["leiden"] = np.array(all_data.obs["leiden"]).astype(int)

In [42]:
# Randomly sample cells from time points / positions / clusters
n_labels = 3
region_properties_table_pos = region_properties_table[
    region_properties_table["position"] == "16"
]
region_properties_table_pos = (
    region_properties_table_pos[region_properties_table_pos["time_point"] >= 169]
    .sample(n=75, replace=False, random_state=8776)
    .copy()
)

# Create dir for Projections of the images to save in
label_dir = f"/{seg_directory}/Annotation_QC_updated_replicates/labels_29_05_23_v22/"
if not os.path.exists(label_dir):
    os.makedirs(label_dir)

for i in tqdm(range(len(region_properties_table_sampled))):
    plot_save(region_properties_table_sampled, i, label_dir)

In [ ]:
plt.rcParams["figure.dpi"] = 450
plt.style.use("dark_background")
# Annotate all structure in jupyter
labeled_DF = create_labels(
    label_dir, os.path.basename(os.path.dirname(label_dir)) + ".csv"
)

# Annotate all structure
# 1 Other structures
# 2 histone
# 3 actin
# 4 Tubulin
# 5 Lamin
# 6 CAAX